In [1]:
from bs4 import BeautifulSoup
import selenium
from selenium import webdriver
from selenium.webdriver.chrome.options import Options as ChromeOptions
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import re
import requests
# from newspaper import Article
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from string import punctuation
from heapq import nlargest
import textwrap as tr
import urllib.request
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
import heapq
import warnings
warnings.filterwarnings('ignore')

In [2]:
# questions

n_pages = (input("please input total number pages for google search results (assuming 10 links per page)"))
                 
query =(input("please enter google search keywords"))
                 
filename = (input("please enter filename to save the results"))
                 


please input total number pages for google search results (assuming 10 links per page)5
please enter google search keywordslisi+block dental review
please enter filename to save the resultslisi15


In [3]:
def convert(url):
    scraped_data = urllib.request.urlopen(url)
    article = scraped_data.read()

    parsed_article = BeautifulSoup(article,'lxml')

    paragraphs = parsed_article.find_all('p')

    article_text = ""

    for p in paragraphs:
        article_text += p.text

    # Removing Square Brackets and Extra Spaces
    article_text = re.sub(r'\[[0-9]*\]', ' ', article_text)
    article_text = re.sub(r'\s+', ' ', article_text)

    # Removing special characters and digits
    formatted_article_text = re.sub('[^a-zA-Z]', ' ', article_text )
    formatted_article_text = re.sub(r'\s+', ' ', formatted_article_text)

    sentence_list = sent_tokenize(article_text)

    stopwords = nltk.corpus.stopwords.words('english')

    word_frequencies = {}
    for word in nltk.word_tokenize(formatted_article_text):
        if word not in stopwords:
            if word not in word_frequencies.keys():
                word_frequencies[word] = 1
            else:
                word_frequencies[word] += 1

    maximum_frequncy = max(word_frequencies.values())

    for word in word_frequencies.keys():
        word_frequencies[word] = (word_frequencies[word]/maximum_frequncy)

    sentence_scores = {}
    for sent in sentence_list:
        for word in nltk.word_tokenize(sent.lower()):
            if word in word_frequencies.keys():
                if len(sent.split(' ')) < 30:
                    if sent not in sentence_scores.keys():
                        sentence_scores[sent] = word_frequencies[word]
                    else:
                        sentence_scores[sent] += word_frequencies[word]

    summary_sentences = heapq.nlargest(7, sentence_scores, key=sentence_scores.get)

    summary = ' '.join(summary_sentences)

    return summary, len(formatted_article_text)

In [4]:
# path = r'C:\Users\StephenHaynesAus\OneDrive - GC Dental\webscraping\chromedriver.exe'
path = r'chromedriver.exe'

s = Service(path)
options=Options()
options.add_argument('--headless')

driver = webdriver.Chrome(service=s, options=options)

In [5]:
# query to obtain links

print("on it........")

# initiate empty list to append results
links = []

# query = 'intext:lisi+block research'

# filename = 'lisi11'

# specify number of pages to load on SERP
n_pages = int(n_pages)
# n_pages = 5

for page in range(1, n_pages):
    # capture 10 links per page
    url = "http://www.google.com/search?q=" + query + "&start=" + str((page - 1)*10)
    driver.get(url)
    # for each entire page
    soup = BeautifulSoup(driver.page_source,'html.parser')
    # look for the specifc class name for "Lisi Block"
    search = soup.find_all('div',class_ = "yuRUbf")
    # get every links in that page
    for g in search:
        links.append(g.a.get('href'))

## convert to 2D list:
n=1
links = [links[i:i+n] for i in range(0,len(links),n)]


on it........


In [6]:
# parse every function and summarize the article

h=1
all_text=[]


for url in links:
    url = ','.join(url)
    if (re.search(r'.pdf',url) or re.search(r'gcamerica',url) or re.search(r'gcaustralasia',url) 
        or re.search(r'europe.gc',url) or re.search(r'gcasia',url) or re.search('gceurope',url) or
       re.search(r'dentalsky',url) or re.search(r'henryschein',url) or re.search(r'kentexpress', url) or
       re.search(r'onlinelibrary',url) or re.search(r'insourceonline',url) or re.search(r'inqmobility',url) or
       re.search(r'dentaltix',url) or re.search(r'issuu',url) or re.search(r'dentalpathway',url) 
        or re.search(r'bfmulholland',url) or re.search(r'dentistryshow',url) or re.search(r'accessdata.fda.gov',url) or
       re.search(r'dentaleducationhub',url) or re.search(r'3di-dental-digital',url) or re.search(r'ur.booksc.eu',url) or
       re.search(r'dentinaltubules',url) or re.search(r'tdsc',url) or re.search(r'pattersondenta',url) or
       re.search(r'eudent',url) or re.search('dentex',url) or re.search(r'ddgroup',url) or re.search(r'basiqdental',url) or
       re.search(r'unident',url) or re.search(r'gacd',url) or re.search(r'pearsondental',url) or
       re.search(r'laboratoire.dentanor',url) or re.search(r'proclinic.es',url) or re.search(r'journalofosseo',url) or
       re.search(r'e-meditrans.pl',url) or re.search(r'unidentdanmark',url) or re.search(r'goodreads',url) or
       re.search(r'alphabond',url) or re.search(r'amazon',url) or re.search(r'dme-medical',url)):
        
        continue
    else:
        try:
            article, length = convert(url)
            if len(article) < 30:
                continue
            else:
                with open(filename+".txt",'a',encoding='utf-8') as f:
                    f.write(f"{h}) Link of article: {url}, {length} words before summary \n\n{tr.fill(article,width=120)}\n\n\n")
                    h+=1
                print(f"length of article is {len(article)}, link: {url}")
                print()
        except:
            print(f'{url} not ok')
            print()
        
                
print("done!")

https://www.dentaleconomics.com/science-tech/article/14277628/pearls-for-your-practice-gc-initial-lisi-block not ok

length of article is 623, link: https://www.dentalproductsreport.com/view/5ws-gc-initial-lisi-block

length of article is 551, link: https://www.nature.com/articles/s41415-021-3601-z

length of article is 894, link: https://www.nature.com/articles/s41415-022-4287-6

https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6902521/ not ok

https://www.sciencedirect.com/science/article/pii/S1013905220307458 not ok

length of article is 1010, link: https://www.dental-tribune.com/c/gc-international/news/interview-cad-cam-materials-play-a-key-role-in-our-research-because-they-are-the-future/

https://www.researchgate.net/figure/Mechanical-properties-of-IPS-emax-press-and-GC-Initial-LiSi-press-materials_tbl1_337852941 not ok

length of article is 116, link: https://www.semanticscholar.org/paper/Fracture-behavior-of-CAD%2FCAM-ceramic-crowns-before-Garoushi-S%C3%A4ilynoja/9656af1dee1d81e616